In [ ]:
from src.papers.domain.multimodal_paper_query import ComparisonOperator, FilterCondition, SourceFilters, CitationFilters, QueryParameters, MultiModalPaperQuery, CitationParameters
from src.papers.io.db import Milvus, Neo4j, SQLite
import os

In [ ]:
MILVUS_COLLECTION = os.getenv("MILVUS_COLLECTION")
MILVUS_ALIAS = os.getenv("MILVUS_ALIAS")
MILVUS_HOST = os.getenv("MILVUS_HOST")
MILVUS_PORT = os.getenv("MILVUS_PORT")
NEO4J_URI = os.getenv("NEO4J_URI")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")
NEO4J_DATABASE = os.getenv("NEO4J_DATABASE")

In [ ]:
milvus_client = Milvus(
        collection=MILVUS_COLLECTION, 
        alias=MILVUS_ALIAS,
        host=MILVUS_HOST,
        port=MILVUS_PORT,
    )   

In [ ]:
neo4j_client = Neo4j(
    uri=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    database=NEO4J_DATABASE
)

In [ ]:
sqlite_client = SQLite("")

In [ ]:
query_client = MultiModalPaperQuery(relational_db_client=sqlite_client, vector_db_client=milvus_client, graph_db_client=neo4j_client)
query_client.query_vector_db(
    "spot_instances"
)


In [ ]:
param = CitationParameters(title="Admission Control for Elastic Cloud Services.", year="2012")
len(query_client.get_citations(parameters=param))

In [ ]:
param.model_dump(mode="json")

In [ ]:
filter_condition = FilterCondition(field = "sda", operator="eq", values=["adsaas"])
citation_filters = CitationFilters(topic="sdasdasdasd", filters=[filter_condition])
QueryParameters(citation_filters=citation_filters)

In [ ]:
filters = [
    FilterCondition(field="Year", operator="eq", values=["2012", "2013", "2014"])
]

source_filters = SourceFilters(text="Spot instances", filters=filters)
query_parameters = QueryParameters(source_filters=source_filters)

In [ ]:
df = query_client.query(query_parameters)

In [ ]:
import polars as pl
df.head(10)

In [ ]:
from src.papers.io.db import SQLite

sql_client = SQLite('../data/RelationalDB/ccgrid.db') 
sql_client.connect()

In [ ]:
results = sql_client.query("""
                SELECT c_inst.year, comm.name, inst.name, inst.country
                FROM Committee_Institution AS c_inst 
                INNER JOIN Committee as comm
                ON comm.id_committee = c_inst.id_committee
                INNER JOIN Institutions as inst
                ON inst.id_institution = c_inst.id_institution
                """)

for result in results:
    print(result)
    
commitees_per_conference = []
for committee in results:
    commitees_per_conference.append({
        "Conference": "ccgrid",
        "Year": committee[0],
        "Committee": {
            "committee_name": committee[1],
            "committee_institution": committee[2],
            "committee_country": committee[3] 
        }
    })
    
print(commitees_per_conference)

In [ ]:
import polars as pl
df = pl.DataFrame(commitees_per_conference)\
    .group_by("Year", "Conference")\
    .agg([pl.col("Committee")])

In [ ]:
df.explode("Committee").unnest("Committee").sql("""SELECT Year, committee_country, COUNT(*) as committee_count
FROM self
GROUP BY Year, committee_country
QUALIFY ROW_NUMBER() OVER (PARTITION BY Year ORDER BY COUNT(*) DESC) = 1
ORDER BY Year, committee_country;""")aws

In [ ]:
df.explode("Committee").unnest("Committee")

In [ ]:
print(df.schema)

In [ ]:
df.explode("Committee").unnest("Committee")

In [ ]:
import src.papers.domain.multimodal_paper_query as mpq
from src.papers.domain.multimodal_paper_query import ComparisonOperator, FilterCondition, SourceFilters, CitationFilters, QueryParameters, MultiModalPaperQuery, CitationParameters
from src.papers.io.db import Milvus, Neo4j, SQLite
import os
import polars as pl

MILVUS_COLLECTION = os.getenv("MILVUS_COLLECTION")
MILVUS_ALIAS = os.getenv("MILVUS_ALIAS")
MILVUS_HOST = os.getenv("MILVUS_HOST")
MILVUS_PORT = os.getenv("MILVUS_PORT")
NEO4J_URI = os.getenv("NEO4J_URI")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")
NEO4J_DATABASE = os.getenv("NEO4J_DATABASE")

milvus_client = Milvus(
        collection=MILVUS_COLLECTION, 
        alias=MILVUS_ALIAS,
        host=MILVUS_HOST,
        port=MILVUS_PORT,
    )   
neo4j_client = Neo4j(
    uri=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    database=NEO4J_DATABASE
)

query_client = mpq.MultiModalPaperQuery(relational_db_client=SQLite, vector_db_client=milvus_client, graph_db_client=neo4j_client)



In [ ]:
query = mpq.QueryParameters(
    source_filters=mpq.SourceFilters(
        text="Spark"
    ),
    citation_filters=mpq.CitationFilters(
        text="Spark"
    )
)

In [ ]:
# If source is provided, start by recovering source paper data.
df_source = query_client.query_source(query.source_filters) if query.source_filters else []

# If citations are provided, make query with its filters
df_target = query_client.query_citation(query.citation_filters) if query.citation_filters else []   

In [ ]:
df_source.select(["source_title", "cited_title"]).to_dicts()

In [ ]:
(
    df_source
    .select([
        'source_title',
        'source_year',
        'source_country',
        'source_authors',
        'source_conference',
        'cited_title',
        'cited_year',
        'cited_predominant_country',
        'cited_conference',
        'cited_authors',
        'cited_country',
        'cited_institution'
    ])
    .join(
        (
            df_target
            .with_columns(
                pl.col("source_title").cast(pl.String),
                pl.col("source_year").cast(pl.String),
                pl.col("cited_title").cast(pl.String),
                pl.col("cited_year").cast(pl.String),
            )
            .select([
                'source_title',
                'source_year',
                'source_predominant_country',
                'source_authors',
                'source_conference',
                'source_institution',
                'source_country',
                'cited_title',
                'cited_year',
                'cited_predominant_country',
                'cited_conference',
                'cited_authors',
                'cited_country',
                'cited_institution'                
            ])
        ), 
        on=["source_title", "source_year", "cited_title", "cited_year"]
    )
).head(100)

In [ ]:

filters = [
    FilterCondition(
        parenthesis=mpq.ParenthesisIndicator.open,
        field="Authors",
        operator="eq",
        values=["Juan"]
    ),
    FilterCondition(
        connector=mpq.LogicConnector.or_op,
        parenthesis=mpq.ParenthesisIndicator.close,
        field="Authors",
        operator=mpq.ComparisonOperator.eq,
        values=["Pablo"]
    ),
    FilterCondition(
        connector=mpq.LogicConnector.and_op,
        field="Year",
        operator=mpq.ComparisonOperator.eq,
        values=["2022"]
    )      
    
]

source_filters = SourceFilters(text="aaaaaa", filters=filters)

In [ ]:
source_filters

In [ ]:
query_client.translate_vector_db_filters(filter=source_filters)

In [ ]:

f"[{", ".join(f'"{value}"' for value in ["aaaa", "bbbb", "cccc"])}]"

In [ ]:
print(mpq.LogicConnector.or_op.value)

In [ ]:
source_filters = SourceFilters(
    text="Spot instances", 
    filters=[
        FilterCondition(
            connector=mpq.LogicConnector.none.value,
            field="Year",
            operator=mpq.ComparisonOperator.eq,
            values=["2022"]
        )
    ]
)
df_result = query_client.query(QueryParameters(source_filters=source_filters))

In [ ]:
from ollama import chat, ChatResponse        

def get_sql_query(query):
    schema = df_result.schema
    SYSTEM_PROMPT = f"""
    You are an AI assistant that translates a given query abou papers into a valid SQL query to a table with the following schema of columns:
    {schema}\n
    """

    USER_PROMPT = query

    llms = [
        "llama3.1:8b",
        "llama3.2",
        "gemma3:12b-it-qat",
        "llama3.1:8b",
        "qwen3"
    ]
    selected_llm = 0
    print(llms[2])
    
    response: ChatResponse = chat(
        model=llms[selected_llm],
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": USER_PROMPT},
        ]
    )
    print(response)
    return response



In [ ]:
print(get_sql_query("""
    What are the most cited papers in year 2022 published by German universities about spot instances            
""").message.content)

# Queries a realizar

Listado de queries a elaborar
1. Obtener papers por año en cada conferencia de publicaciones asiáticas -> Hecho con filtro
2. Distribución por continentes de cada paper aceptado -> Se hace con acumulado
3. Diversidad por continente de los miembros del comité de cada conferencia
4. Comparación por paper de los continentes de los comités de cada conferencia 
5. Cálculo de índice de Gini-Simpson


In [1]:
import src.papers.domain.multimodal_paper_query as mpq
from src.papers.domain.multimodal_paper_query import Conference, FilterCondition
from src.papers.io.db import Milvus, Neo4j, SQLite
import os
import polars as pl
from pydantic import BaseModel, Field
from typing import Optional
from enum import Enum
import pycountry_convert as pc

MILVUS_COLLECTION = os.getenv("MILVUS_COLLECTION")
MILVUS_ALIAS = os.getenv("MILVUS_ALIAS")
MILVUS_HOST = os.getenv("MILVUS_HOST")
MILVUS_PORT = os.getenv("MILVUS_PORT")
NEO4J_URI = os.getenv("NEO4J_URI")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")
NEO4J_DATABASE = os.getenv("NEO4J_DATABASE")

milvus_client = Milvus(
        collection=MILVUS_COLLECTION, 
        alias=MILVUS_ALIAS,
        host=MILVUS_HOST,
        port=MILVUS_PORT,
    )   
neo4j_client = Neo4j(
    uri=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    database=NEO4J_DATABASE
)

query_client = mpq.MultiModalPaperQuery(relational_db_client=SQLite, vector_db_client=milvus_client, graph_db_client=neo4j_client)

/home/luis-garcia/projects/tfm/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [28]:
class Conference(str, Enum):
    """
    Supported conferences for papers
    """
    ccgrid = "ccgrid"
    eurosys = "eurosys"

class Continent(str, Enum):
    """
    Continent codes and their description
    """    
    AS = 'Asia',
    EU = 'Europe',
    NA = 'North America',
    SA = 'South America',
    AF = 'Africa',
    OC = 'Oceania',
    AN = 'Antarctica'

class PapersPerConferenceContinentYearParameters(BaseModel):
    """
    Parameters to query papers per conference, continent and year
    """
    conferences: Optional[list[Conference]] = Field(default=None, description="Conference to use as filter")
    years: Optional[list[str]] = Field(default=None, description="Years to use as filter")
    continents: Optional[list[str]] = Field(default=None, description="Continents to filter by")

class PapersPerConferenceAndContinent(BaseModel):
    """
    Parameters to query papers per conference and continent
    """
    conferences: Optional[list[Conference]] = Field(default=None, description="Conference to use as filter")
    continents: Optional[list[str]] = Field(default=None, description="Continents to filter by")    

# Distribución por continentes
def query_paper_count_per_conference_continent_and_year(parameters: PapersPerConferenceContinentYearParameters) -> pl.DataFrame:  
    """
    Method to query papers published per conference and continent
    Args:
        parameters (CitationsPerContinentParameters): _description_

    Returns:
        pl.DataFrame: Dataframe with calculated number of papers published in each conference,
                    year and continent
    """
    filter_conditions = []
    if parameters.conferences and len(parameters.conferences) > 0:
        filter_conditions.append(
            FilterCondition(
                connector=mpq.LogicConnector.none.value,
                field="Conference",
                operator=mpq.ComparisonOperator.eq.value,
                values=parameters.conferences
            )
        )
    
    if parameters.years and len(parameters.years) > 0:
        filter_conditions.append(
            FilterCondition(
                connector=mpq.LogicConnector.and_op.value if len(filter_conditions) > 0 else mpq.LogicConnector.none.value,
                field="Year",
                operator=mpq.ComparisonOperator.eq.value,
                values=parameters.years
            )
        )
             
    query_parameters = mpq.QueryParameters(
        source_filters=mpq.SourceFilters(
            text=None,
            filters=filter_conditions if len(filter_conditions) > 0 else None
        )
    )
    
    df_query = query_client.query(query_parameters)        
    df_result = (
        df_query
        .select([
            "source_title",
            "source_year",
            "source_predominant_continent",
            "source_conference"
        ])
        .explode("source_predominant_continent")        
        .filter(
            pl.when(parameters.continents is not None)
            .then(pl.col("source_predominant_continent").is_in(parameters.continents))
            .otherwise(True)            
        )
        .unique()
        .group_by("source_year", "source_conference", "source_predominant_continent")
        .agg([
            pl.len().alias("paper_count")
        ])
        .sort("source_conference","source_year", "paper_count", descending=True)
    )
    
    return df_result

def query_paper_count_per_conference_and_continent(parameters: PapersPerConferenceAndContinent) -> pl.DataFrame:
    
    parameters_per_year = PapersPerConferenceContinentYearParameters(
        conferences=parameters.conferences, 
        continents=parameters.continents,    
    )
    df_result = (
        query_paper_count_per_conference_continent_and_year(parameters_per_year)
        .group_by("source_conference", "source_predominant_continent")
        .agg([
            pl.sum("paper_count")
        ])
        .sort("source_conference", "paper_count", descending=True)
    )
    return df_result
    
    

In [ ]:
# df_result = query_paper_count_per_conference_and_continent(CitationsPerContinentParameters(years=["2020", "2021", "2022"], conferences=["ccgrid"], continents=["Asia", "Europe"]))
df_result = query_paper_count_per_conference_and_continent(
    PapersPerConferenceAndContinent(
        conferences=["ccgrid"], 
        # years=["2020"]
        continents=["Asia", "Europe"]
    )
)
df_result.head(15)

In [ ]:
# Papers por año de cada conferencia de publicaciones por continente país

def get_continent(country_code):
    try:
        continent_code = pc.country_alpha2_to_continent_code(country_code)
        continent_name = pc.convert_continent_code_to_continent_name(continent_code)
        return continent_name
    except:
        return "Unknown"
    
def get_committees_per_conference_country_year(
    conferences: Optional[list[Conference]]=None,
    years: Optional[list[str]]=None
) -> pl.DataFrame:
    df_query = query_client.get_conference_committees(conferences)
    df_result = (
        df_query
        # .filter(pl.col("source_predominant_continent").is_in(conferences))
        .unnest("committee")
        .group_by("conference", "year", "committee_country")
        .agg([
            pl.len().alias("committee_count")
        ])
        .sort("conference", "year", "committee_count", descending=True)
    )
    return df_result

def get_committees_per_continent(
    conferences: Optional[list[Conference]]=None, 
    continents: Optional[list[Continent]]=None
) -> pl.DataFrame:
    df_query = get_committees_per_year(
        conferences=conferences
    )
    
    df_result = (
        df_query
        .with_columns([
            pl.col("committee_country")
            .map_elements(get_continent, return_dtype=pl.String)
            .alias("continent")
        ])
        .filter(
            pl.when(continents is not None)
            .then(pl.col("continent").is_in(continents))
            .otherwise(True)            
        )        
        .group_by("conference", "year", "continent")
        .agg([
            pl.sum("committee_count").alias("committee_count")
        ])
        .sort("conference", "year", "committee_count", descending=True)        
    )
    return df_result

In [45]:
get_committees_per_continent(continents=["North America", "Asia"])


conference,year,continent,committee_count
str,str,str,u32
"""sigcomm""","""2023""","""North America""",39
"""sigcomm""","""2023""","""Asia""",12
"""sigcomm""","""2022""","""North America""",45
"""sigcomm""","""2022""","""Asia""",5
"""sigcomm""","""2021""","""North America""",41
…,…,…,…
"""IEEEcloud""","""2014""","""Asia""",15
"""IEEEcloud""","""2013""","""North America""",58
"""IEEEcloud""","""2013""","""Asia""",17


In [ ]:
Continent.AN.vale

'Antarctica'

In [ ]:
from enum import Enum
class Country(str, Enum):
    """
    Supported conferences for papers
    """
    JP = "JP"
    eurosys = "eurosys"
countries = ['JP', 'NL', 'CN', 'US', 'FR', 'DE']
asian_countries = []
asian_from_list = [print(country) for country in countries if country in _countries or not asian_countries]

In [ ]:
from enum import Enum
class Conference(str, Enum):
    """
    Supported conferences for papers
    """
    ccgrid = "ccgrid"
    eurosys = "eurosys"
    
[print(conference.value) for conference in Conference]

In [ ]:
[k for k in {"asdas": "asdasd", "qqq": "dasdas"}]